In [4]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt

In [5]:
def nonlin(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

In [6]:
def run(a1,y,alpha,n_neurons,la=0,niter=1000):
    syn = [2*np.random.random((2,n_neurons[0])) - 1 ]
    i=0
    for i in range(len(n_neurons)-1):
        syn.append(2*np.random.random((n_neurons[i],n_neurons[i+1])) - 1)
    if len(n_neurons)>1:
        i+=1

    syn.append(2*np.random.random((n_neurons[i],1)) - 1)

   
    ns=len(syn)
    ntest=len(y)
    error1=np.zeros((niter))

    for i in range(niter):
        der = []
        for k in  range(ns):
            der.append(np.zeros(syn[k].shape))
        #print  i
        for j in range(ntest):#loop on all the observations from the test sample
            # forward propagation
            l=[np.array([a1[j,:]])] 

            for k in range(ns):
                l.append(nonlin(np.dot(l[-1],syn[k])))
            

            err=l[-1]-y[j]
            error1[i]+= np.dot(err,err)

            # backpropagation
            D= err*nonlin(l[-1],deriv=True)
            
            for k in range(ns-1,-1,-1):
                der[k] +=l[k].T.dot(D)
                D = D.dot(syn[k].T)*nonlin(l[k],deriv=True)
                
        for k in range(ns):
            syn[k]-=alpha*(der[k]/ntest + la*syn[k])

    a=a1
    # evaluation
    # forward propagation
    for k in range(ns):
        z = np.matmul(a,syn[k])
        a = nonlin(z)
    
    return np.sqrt(error1),a,syn

In [ ]:
# training test
a1 = np.array([[1.,1.],[0.,0.],[1.,0.],[0.,1.]])
y = np.array([[0.],[0.],[1.],[1.]])

n_neurons=[3]

s=run(a1,y,10.,n_neurons,niter=10000)

In [58]:
# Input test
ntest=1000
x=np.random.rand(ntest,2)

y=x[:,0]**2+x[:,1]**2

X=x
mix1=np.min(X[:,0])
max1=np.max(X[:,0])

X[:,0]= (X[:,0]-mix1)/(max1-mix1)
mix2=np.min(X[:,1])
max2=np.max(X[:,1])

X[:,1]= (X[:,1]-mix2)/(max2-mix2)


miy=np.min(y)
may=np.max(y)

Y= np.array([(y-miy)/(may-miy)]).T



In [ ]:
n_neurons=[7,3,2]
s=run(X,Y,1.,n_neurons,niter=1000)

In [ ]:
plt.plot(s[1])
plt.xlabel('Iter')
plt.ylabel('Error')
plt.show()

In [ ]:
# new data
xdat=np.linspace(0,1,10)
xdat=np.array([xdat,xdat]).T
ydat=xdat[:,0]**2+xdat[:,1]**2 #True value
xdat_rescaled=np.array([(xdat[:,0]-mix1)/(max1-mix1),(xdat[:,1]-mix2)/(max2-mix2)]).T

In [ ]:
syn=s[2]
a=xdat_rescaled

# predicition
for k in range(len(syn)):
    z = np.matmul(a,syn[k])
    a = nonlin(z)
ypred= a*(may-miy)+miy
plt.plot(xdat[:,0],ydat,xdat[:,0],ypred)
plt.legend(['True value','Model'])
plt.show()